# Import

In [10]:
import os
import torch
import shutil
from pathlib import Path

from datasets import load_dataset 
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

# Setting

In [12]:
MODEL_ID = "./base_model"     
OUT_DIR  = "./model"          

DATASET_ID = "LGAI-EXAONE/MANTA-1M"
DATASET_SPLIT = "train"

NUM_CALIBRATION_SAMPLES = 256
MAX_SEQUENCE_LENGTH = 512

# Quantization
SCHEME = "FP8"
TARGETS = ["Linear"]
IGNORE  = ["embed_tokens", "lm_head"]

# Model Loads

In [13]:
print("[INFO] 모델 로드 중...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
)

print("[INFO] 모델/토크나이저 로드 완료")

[INFO] 모델 로드 중...
[INFO] 모델/토크나이저 로드 완료


# Dataset Loads & Preprocess

In [14]:
print("[INFO] 캘리브레이션 데이터 로드 중...")

ds = load_dataset(
    DATASET_ID,
    split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]",
)

def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False)
    }

ds = ds.map(preprocess)

print("[INFO] 데이터 전처리 완료")

[INFO] 캘리브레이션 데이터 로드 중...
[INFO] 데이터 전처리 완료


# GPTQ Quantization

In [ ]:
print(f"[INFO] GPTQ 시작 (scheme={SCHEME}, samples={NUM_CALIBRATION_SAMPLES}, max_len={MAX_SEQUENCE_LENGTH})...")

recipe = [
    GPTQModifier(
        scheme=SCHEME,
        targets=TARGETS,
        ignore=IGNORE,
    )
]


oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

print("[INFO] GPTQ 완료")

[INFO] GPTQ 시작 (scheme=FP8, samples=256, max_len=512)...


Tokenizing: 100%|██████████| 256/256 [00:00<00:00, 835.30 examples/s]

2026-02-06T17:38:24.707907+0900 | reset | INFO - Compression lifecycle reset
2026-02-06T17:38:24.711376+0900 | from_modifiers | INFO - Creating recipe from modifiers
2026-02-06T17:38:24.748103+0900 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2026-02-06T17:38:24.749047+0900 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


2026-02-06T17:38:25.120863+0900 | dispatch_for_sequential | WARNING - CUDA/XPU is not available! Compressing model on CPU instead


(1/31): Calibrating: 100%|██████████| 256/256 [01:24<00:00,  3.03it/s]

...

(30/31): Calibrating: 100%|██████████| 256/256 [01:18<00:00,  3.25it/s]

2026-02-06T18:47:55.744044+0900 | compress_modules | INFO - Quantizing model.layers.29.self_attn.q_proj using 256 samples


2026-02-06T18:47:56.075915+0900 | compress | METRIC - time 0.33s
2026-02-06T18:47:56.076295+0900 | compress | METRIC - error 192.78
2026-02-06T18:47:56.076869+0900 | get_GPU_usage_nv | WARNING - Pynml library error:
 NVML Shared Library Not Found
2026-02-06T18:47:56.077110+0900 | compress | METRIC - Compressed module size: 8.388614 MB
2026-02-06T18:47:56.078047+0900 | compress_modules | INFO - Quantizing model.layers.29.self_attn.k_proj using 256 samples
2026-02-06T18:47:56.292487+0900 | compress | METRIC - time 0.21s
2026-02-06T18:47:56.292887+0900 | compress | METRIC - error 51.08
2026-02-06T18:47:56.293512+0900 | get_GPU_usage_nv | WARNING - Pynml library error:
 NVML Shared Library Not Found
2026-02-06T18:47:56.293731+0900 | compress | METRIC - Compressed module size: 2.097158 MB
2026-02-06T18:47:56.294223+0900 | compress_modules | INFO - Quantizing model.layers.29.self_attn.v_proj using 256 samples
2026-02-06T18:47:56.497794+0900 | compress | METRIC - time 0.20s
2026-02-06T18:47:5

(31/31): Propagating: 100%|██████████| 256/256 [00:00<00:00, 2099.03it/s]

2026-02-06T18:48:55.795004+0900 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2026-02-06T18:48:55.800830+0900 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
[INFO] GPTQ 완료


# Model Save

In [16]:
os.makedirs(OUT_DIR, exist_ok=True)

model.save_pretrained(OUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUT_DIR)

print(f"[INFO] 모델 저장 완료: {OUT_DIR}")

2026-02-06T18:49:12.588411+0900 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 210it [00:01, 152.34it/s]


[INFO] 모델 저장 완료: ./model


# Submission

In [17]:
zip_name = "baseline_submit"
print(f"[INFO] {zip_name}.zip 생성 중...")

shutil.make_archive(
    base_name=zip_name,
    format="zip",
    root_dir=".",
    base_dir=OUT_DIR,
)

print(f"[INFO] 생성 완료: {zip_name}.zip")

[INFO] baseline_submit.zip 생성 중...
[INFO] 생성 완료: baseline_submit.zip
